In [13]:
import matplotlib.pyplot as plt
import numpy as np
from vpython import *
from sympy import *
def column(matrix, i):
    return [row[i] for row in matrix]

In [25]:
def alexpolynomial(x,y,z):
    points = np.zeros(shape=(len(x),3))
    points[:,0]=x
    points[:,1]=y
    points[:,2]=z

    #Remove extraneous 3D points
    deletes = []
    for i in range(0,len(points)-1):
        if (points[i][0]==points[i+1][0]) and (points[i][1]==points[i+1][1]):
            deletes.append(i+1)       
    project = np.delete(points, deletes, axis=0)

    #Finding the corssing points
    num = 0
    oindex = []
    uindex = []
    for i in range(1,len(project)-1):
        point1 = project[i]
        for j in range(i+1,len(project)):
            point2 = project[j]
            if (point1[0]==point2[0]) and (point1[1]==point2[1]):
                num = num + 1
                if (point1[2]>point2[2]):
                    oindex.append(i)
                    uindex.append(j)
                else:
                    oindex.append(j)
                    uindex.append(i)

    project = np.delete(project, 2, axis=1)   #reduce to the string to a 2D projection             
    t = symbols('t')
    M = zeros(num, num) #Create empty matrix for alexander polynomial

    #Iterate over the number of crossings
    for i in range(num):
        over = project[int(oindex[i]+1)] - project[int(oindex[i])]
        under = project[int(uindex[i]+1)] - project[int(uindex[i])]
        if np.cross(over,under) == -1:
            pol = [1-t, t, -1] #order is: [overstrand arc, understrand head arc, understrand tail arc]
        else:
            pol = [1-t, -1, t]

        endarc = [a-1 for a in uindex] #define the arc by the end of each arc, i.e. the point where it is cut off 
                                    #by the overstrand at the crossing. Then the 'name' of the arc is the index.

        #for overstrand arc
        list1 = endarc + [oindex[i]]
        list1 = sorted(list1)
        arc1 = list1.index(int(oindex[i]))
        if arc1 > (num-1):
            arc1 = 0
        M[i*(num) + arc1] = pol[0]

        #for understrand head arc
        list2 = endarc + [uindex[i]+1]
        list2 = sorted(list2)
        arc2 = list2.index(int(uindex[i]+1))
        if arc2 > (len(endarc)-1):
            arc2 = 0
        M[i*(num) + arc2] = pol[1]

        #for understrand tail arc
        list3 = sorted(endarc)
        arc3 = list3.index(int(uindex[i]-1))
        M[i*(num) + arc3] = pol[2]

    M.col_del(0)
    M.row_del(0)
    return M.det()

In [26]:
(x,y,z) = np.loadtxt('/Users/Keerthana/Downloads/Trefoil.csv', delimiter=',', unpack=True)
alexpolynomial(x,y,z)

-t**2 + t - 1

In [27]:
(x,y,z) = np.loadtxt('/Users/Keerthana/Downloads/Figure 8.csv', delimiter=',', unpack=True)
alexpolynomial(x,y,z)

-t**2 + 3*t - 1